In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchaudio
import numpy as np
import scipy
import sklearn
import pandas as pd
import PIL
import matplotlib
import tokenizers
import datasets
import transformers
import matplotlib.pyplot as plt
from collections import Counter
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


# Data exploration

Here we explore each feature column and comment on how we are going to preprocess each one and what we might consider doing in the future. We have a total of 140700 samples. The last feature is the label depression which we will train on.

## Loading the dataset

In [ ]:
# Load the datasets
data_file = 'data/train.csv'
test_file = 'data/test.csv'

data = pd.read_csv(data_file)
test = pd.read_csv(test_file)

print(f"Samples in training set {data.shape[0]}")
print(f"Samples in test set: {test.shape[0]}")

# id Column

In [ ]:
# Print examples of the 'id' column
print("Examples of 'id' column:")
print(data["id"].head())

print("")

print("We have " ,data["id"].count()," amount of samples")

We will drop the id column since there is no correlation between this and the label.

## Name column

In [ ]:
# Count occurrences of each unique name
name_counts = data["Name"].value_counts()

# Print results
print("Unique names sorted by count (most to least):")
for name, count in name_counts.items():
    print(f"{name}: {count}")

print(f"Total unique names: {len(name_counts)}")


We thought about dropping the name column since we thought that this may not have any correlation to the label. But we realised that there might be a correlation, so we decided to keep it. The correlation being that for example having a "unattractive" name can affect your life in a bad way. 


For pre-processing this we will to give a value of how unique it is in the dataset, so for now we change the name with how frequent it is.


Things to consider:

Find a way to "rate" each name instead of how frequent the name is. Impute or change wrong names to missing name.

## Gender

In [ ]:
# Print examples of the 'Name' column
print("Examples of 'Name' column:")
print(data["Gender"].head())

print("")

print("Number of unique genders:", data["Gender"].nunique())

We will just encode male to 1 and female to 0.

## Age

In [ ]:
print("Examples of 'Age' column:")
print(data["Age"].head())

print("")

print("Number of unique ages:", data["Age"].nunique())

We will only change the number from float to integer.

## City

In [ ]:
print("Examples of 'City' column:")
print(data["City"].head())

print("")

print("Number of unique cities:", data["Age"].nunique())

In [ ]:
# Count occurrences of each unique city
city_counts = data["City"].value_counts()

# Find first occurrence index for each unique city
first_occurrence = data.groupby("City").apply(lambda x: x.index[0])

# Sort cities by count in descending order
sorted_cities = city_counts.index  # Cities sorted by count (default sorting from most to least)

count = 0
# Print results
print("Unique cities sorted by count (most to least):")
for city in sorted_cities:
    if city_counts[city] > 0:
        count += 1
        print(f"{city}: First Index = {first_occurrence[city]}, Count = {city_counts[city]}")

print(f"Total unique cities: {count}")


We found population, density, literacy and sex ratio for each major city in our dataset. We then merged this in our data and removed city column. For the minor cities or the wrongly written cities we took the average of the other columns.


To consider:

We should consider adjusting the imputing of the minor and wrongly written cities to for example lower than average.

# Working Professional or Student

In [ ]:
print("Examples of 'Working Professional or Student' column:")
print(data["Working Professional or Student"].head())

print("")

print("Number of unique Working Professional or Student:", data["Working Professional or Student"].nunique())

In this column we will change "working professional" to 0 and "student" to 1.

# Profession

In [ ]:
print("Examples of 'Profession' column:")
print(data["Profession"].head())
print("")
print("NaN is student:")
print(data["Profession"][2])
print(data["Working Professional or Student"][2])

print("")

print("Number of unique Profession:", data["Profession"].nunique())



There are occurences of NaN in this column, this happens when the sample is a student. We find it reasonable to insert "Student" in those slots. there are also occurences of NaN on samples that are not students, here we will insert "Missing Profession"

# Academic Pressure

In [ ]:
print("Examples of 'Academic pressure' column:")
print(data["Academic Pressure"].head())

print("")

print("Number of unique Academic pressures:", data["Academic Pressure"].nunique())

For the "Academic Pressure" column we want to replace NaN with zeros.
We are aslo considering merging this feature with work pressure, since they complete eachother. but for now we will have it like this and maybe change it for improving the model later.

# Work pressure

In [ ]:
print("Examples of 'Work Pressure' column:")
print(data["Work Pressure"].head())

print("")

print("Number of unique Work Pressure:", data["Work Pressure"].unique())

We assume that a lower number means less work pressure, therefore We will change Nan to 0 because it is the students that has NaN on Work Pressure

# CGPA

In [ ]:
print("Examples of 'CGPA' column:")
print(data["CGPA"].head())
print(data["CGPA"].mean())

We consider changing the NaN slots to the average of the dataset, which is 7.66

# Study Satisfaction and Job Satisfaction (Satisfaction)

In [ ]:
print("Examples of 'Study Satisfaction' and 'Job Satisfaction' column:")
print(data["Study Satisfaction"].head())
print(data["Job Satisfaction"].head())

You can tell that when there is missing a value in the study satisfaction column, there is a value in the same sample but on the job satisfaction problem. these two columns complete eachother, so we will combine these two columns into one "satisfaction" column.

# Sleep Duration

In [ ]:
sleep_counts = data["Sleep Duration"].value_counts()

print("Unique sleep sorted by count (most to least):")
for e, (duration, count) in enumerate(sleep_counts.items()):
    print(duration, count)
    if e == 6:
        break

This one looks a bit tricky. since there are so few occurnces of other than the four most common inputs in this feature, we will change the numbers to a scale from 1 to 4, where 1 is "less than 5 hours" all the way to 4 which is "more than 8 hours". all the others will be set to 1.

# Dietary Habits

In [ ]:
diet_counts = data["Dietary Habits"].value_counts()

print("Unique sleep sorted by count (most to least):")
for e, (diet, count) in enumerate(diet_counts.items()):
    print(diet, count)
    if e == 6:
        break

* Healthy -> 2
* Moderate -> 1
* Unhealthy -> 0
* The rest -> 1

# Degree

In [ ]:
degree_counts = data["Degree"].value_counts()
print(data["Degree"].nunique())

print("Unique sleep sorted by count (most to least):")
for e, (degree, count) in enumerate(degree_counts.items()):
    print(degree, count)
    if e == 10:
        break

This one is particularly difficult because there are 115 unique degrees and there are not just a few degrees that covers the majority of the dataset either as the case is in the dietary habit feature. Our approach here is to somehow categorize the different degrees into bachelor, master, doctrine etc. and then give each of them a number from -1 to 4 based on the rank of the degree, going from "other" (which is the case where we can't define what degree it is) to professional.

# Have you ever had suicidal thoughts ?

In [ ]:
print("Examples of 'Have you ever had suicidal thoughts ?' column:")
print(data["Have you ever had suicidal thoughts ?"].head())

print("")

print("Number of unique Have you ever had suicidal thoughts ?:", data["Have you ever had suicidal thoughts ?"].nunique())

We will also change this column to binary no / yes to 0 / 1

# Work Study Hours (No prerocessing needed here)

In [ ]:
ws_counts = data["Work/Study Hours"].value_counts()
print(data["Work/Study Hours"].nunique())

print("Unique Work/Study Hours sorted by count (most to least):")
for e, (ws, count) in enumerate(ws_counts.items()):
    print(ws, count)
    if e == 20:
        break

# Financial Stress

In [ ]:
finans_count = data["Financial Stress"].value_counts()
print(data["Financial Stress"].nunique())

print("Unique sleep sorted by count (most to least):")
for e, (fn, count) in enumerate(finans_count.items()):
    print(fn, count)
    if e == 10:
        break

Has a few missing values, we will impute the mean in these.

# Family History of Mental Illness

In [ ]:
print("Examples of 'Family History of Mental Illness' column:")
print(data["Family History of Mental Illness"].head())

print("")

print("Number of unique Family History of Mental Illnesses:", data["Family History of Mental Illness"].nunique())

For this column we will change yes and no to 1 and 0

# Depression - The label
Plotting the distribution of the labels

In [ ]:
target_counts = data['Depression'].value_counts()
target_counts.plot(kind='bar', color=['pink', 'black'])
plt.xticks(ticks=[0, 1], labels=['Not Depressed', 'Depressed'], rotation=0)
plt.title('Distribution of Not Depressed vs Depressed')
plt.show()

You can tell that there is a significant imbalance between depressed and not depressed. 

# Pre-processing

Dropping the Id column

In [ ]:
def dropId(data):
    data = data.drop(columns=["id"])
    return data

Encoding the name column, Replacing each name with its frequency in the dataset

In [ ]:
def name_freq(data):
    data["Name"] = data["Name"].map(data["Name"].value_counts())
    return data

Encoding gender column 0/1

In [ ]:
def gender_encode(data):
    data["Gender"] = data["Gender"].map({"Female": 0, "Male": 1 })
    return data

Age, changing the type

In [ ]:
def age_integer(data):
    data["Age"] = data["Age"].astype(int)
    return data

Academic Preessure, inserting 0 in the empty slots

In [ ]:
def academic_pressure(data):
    data["Academic Pressure"] = np.where(
        data["Academic Pressure"].isnull(), 0,
        data["Academic Pressure"]
    )
    return data

Family History of Mental Illnes. -> 0/1

In [ ]:
def family_mental_illness(data):
    data["Family History of Mental Illness"] = np.where(
        data["Family History of Mental Illness"] == "No", 0,
        np.where(
            data["Family History of Mental Illness"] == "Yes", 1,
            data["Family History of Mental Illness"]
        )
    )
    return data

Have you ever had suicidal thoughts? -> 0/1

In [ ]:
def suicidal_thoughts(data):
    data["Have you ever had suicidal thoughts ?"] = np.where(
        data["Have you ever had suicidal thoughts ?"] == "No", 0,
        np.where(
            data["Have you ever had suicidal thoughts ?"] == "Yes", 1,
            data["Have you ever had suicidal thoughts ?"]
        )
    )
    return data

Financial stress

In [ ]:
def financial_stress(data):
    data["Financial Stress"] = np.where(
        data["Financial Stress"].isnull(), round(data["Financial Stress"].mean()), data["Financial Stress"]
    )
    return data

Working Professional or Student. -> 0/1

In [ ]:
def working_or_student(data):
    data["Working Professional or Student"] = np.where(
        data["Working Professional or Student"] == "Working Professional", 0,
        np.where(
            data["Working Professional or Student"] == "Student", 1,
            data["Working Professional or Student"]
        )
    )       
    return data

Profession. Inserting Student in the empty slots if the sample is a student and missing profession if it is empty and the sample is not a student. also inserting "Missing Profession" in the professions that has less than 10 occurrences in the dataset because the majority of these are wrong information.

In [ ]:
def profession(data):
    # Count occurrences of each profession
    profession_counts = data["Profession"].value_counts()
    
    # Update the "Profession" column
    data["Profession"] = np.where(
        data["Profession"].isnull() & (data["Working Professional or Student"] == 1),
        "Student",
        np.where(
            data["Profession"].isnull() & (data["Working Professional or Student"] == 0), 
            "Missing Profession", 
            np.where(
                data["Profession"].map(profession_counts) < 6,  # If occurrence < 10, set to "Missing Profession"
                "Missing Profession",
                data["Profession"]
            )
        )
    )
    return data


Categorizing Professions and giving each category a number. the number is a rating from 0 to 10 on how happy people are in given categories. We used chatGPT to rate them based on different logical points. see professions_mapping dictionary for description of the ratings.

In [ ]:
def categorize_profession(profession):
    if pd.isna(profession) or "missing" in profession.lower():
        return "Unknown"
    
    profession = profession.lower().strip()

    # Technology
    tech_keywords = ["software", "data scientist", "ux/ui", "developer", "engineer"]
    if any(keyword in profession for keyword in tech_keywords):
        return "Technology"

    # Finance
    finance_keywords = ["accountant", "financial", "investment", "banker", "analyst"]
    if any(keyword in profession for keyword in finance_keywords):
        return "Finance"

    # Healthcare
    healthcare_keywords = ["doctor", "pharmacist", "dentist", "nurse"]
    if any(keyword in profession for keyword in healthcare_keywords):
        return "Healthcare"

    # Education
    education_keywords = ["teacher", "professor", "educational"]
    if any(keyword in profession for keyword in education_keywords):
        return "Education"

    # Engineering
    engineering_keywords = ["civil engineer", "mechanical engineer", "architect"]
    if any(keyword in profession for keyword in engineering_keywords):
        return "Engineering"

    # Marketing & Sales
    marketing_keywords = ["marketing", "sales", "digital marketer", "content writer"]
    if any(keyword in profession for keyword in marketing_keywords):
        return "Marketing/Sales"

    # Trade & Manual Work
    trade_keywords = ["electrician", "plumber", "chef", "mechanic"]
    if any(keyword in profession for keyword in trade_keywords):
        return "Trade"

    # Legal
    legal_keywords = ["lawyer", "judge", "legal"]
    if any(keyword in profession for keyword in legal_keywords):
        return "Legal"

    # Consulting
    consulting_keywords = ["consultant", "business analyst"]
    if any(keyword in profession for keyword in consulting_keywords):
        return "Consulting"

    # Other / Unknown
    return "Other"

def Prefession_categorization(data):
    # Apply categorization
    data["Profession_Category"] = data["Profession"].apply(categorize_profession)

    # Encoding Options

    ## Option 1: Ordinal Encoding (useful if there's a natural order)
    profession_mapping = {
    "Unknown": 0,           # No data on job satisfaction
    "Other": 4,             # Mixed bag, depends on the job
    "Trade": 6,             # Skilled trades often provide satisfaction, but physically demanding
    "Marketing/Sales": 5,   # High variability, stressful but rewarding
    "Consulting": 5,        # High pay but often high stress & long hours
    "Education": 7,         # Fulfilling, but pay can be low and stress can be high
    "Finance": 6,           # High pay, but long hours and stress
    "Engineering": 7,       # Good pay, problem-solving, but deadlines can be stressful
    "Healthcare": 6,        # Rewarding but high stress and burnout risk
    "Legal": 5,             # High salary, but stressful and demanding
    "Technology": 8         # High salary, flexibility, remote work options
}


    data["Profession_Encoded"] = data["Profession_Category"].map(profession_mapping)

    data["Profession"] = data["Profession_Encoded"]

    data = data.drop(columns=["Profession_Encoded", "Profession_Category"])
    return data
    ## Option 2: One-Hot Encoding (better for categorical data)
    # data = pd.get_dummies(data, columns=["Profession_Category"], prefix="Profession")


Work Pressure

In [ ]:
def work_pressure(data):
    data["Work Pressure"] = np.where(
        data["Work Pressure"].isnull(), 0.0, data["Work Pressure"]
    )
    return data

CGPA

In [ ]:
def cgpa(data):
    data["CGPA"] = np.where(
        data["CGPA"].isnull(), round(data["CGPA"].mean(), 2), data["CGPA"]
    )
    return data

Study- and Job Satisfaction

In [ ]:
def satisfaction(data):
    satisfaction = np.where(
        data["Study Satisfaction"].notnull(), data["Study Satisfaction"], np.where(
            data["Job Satisfaction"].notnull(), data["Job Satisfaction"], 0
        )
    )
    data["Job Satisfaction"] = satisfaction
    data = data.drop(columns="Study Satisfaction")
    data.rename(columns={"Job Satisfaction": "Satisfaction"}, inplace=True)
    return data

Sleep Duration

In [ ]:
def sleep_duration(data):
    data["Sleep Duration"] = np.where(
        data["Sleep Duration"] == "Less than 5 hours", 1, np.where(
            data["Sleep Duration"] == "5-6 hours", 2, np.where(
                data["Sleep Duration"] == "7-8 hours", 3, np.where(
                    data["Sleep Duration"] == "More than 8 hours", 4, 1
                )
            )
        )
    )
    return data

Dietary Habits

In [ ]:
def diet(data):
    data["Dietary Habits"] = np.where(
        data["Dietary Habits"] == "Healthy", 2, np.where(
            data["Dietary Habits"] == "Moderate", 1, np.where(
                data["Dietary Habits"] == "Unhealthy", 0, 1
            )
        )
    )
    return data

In [ ]:
def categorize_degree(degree):

    """
    Categorizes the differnet degrees into Schoo, Bachelors, Masters, Doctrate, professional and other
    """
    
    degree = str(degree).strip().lower()

    # School-Level
    if "class 11" in degree or "class 12" in degree:
        return "School"

    # Bachelor's Degrees
    bachelor_keywords = ["b.", "b ", "bachelor", "bcom", "bsc", "btech", "be", "bba", "bca", "ba", "b.ed", "b.arch"]
    if any(keyword in degree for keyword in bachelor_keywords):
        return "Bachelors"

    # Master's Degrees
    master_keywords = ["m.", "m ", "master", "mba", "mcom", "msc", "mtech", "me", "mca", "m.ed", "mpharm", "m.arch"]
    if any(keyword in degree for keyword in master_keywords):
        return "Masters"

    # Doctoral Degrees
    if "phd" in degree:
        return "Doctorate"

    # Professional Degrees
    professional_keywords = ["mbbs", "md", "llb", "llm"]
    if any(keyword in degree for keyword in professional_keywords):
        return "Professional"

    # Unknown / Noisy Data
    return "Other"

def degree(data):
    
    # Apply categorization
    data["Degree"] = data["Degree"].apply(categorize_degree)


    # Define an ordinal mapping
    degree_mapping = {
        "School": 1,
        "Bachelors": 2,
        "Masters": 3,
        "Professional": 4,
        "Doctorate": 5,
        "Other": 0  # Keep 'Other' at the highest level or remove it depending on the approach
    }

    # Apply mapping
    data["Degree"] = data["Degree"].map(degree_mapping)

    return data 



City

In [ ]:
# Load city data
city_data_path = "./Data/indian_cities_data.csv"
df_cities = pd.read_csv(city_data_path)

def city_one_hot(data):
    # Find the index of the "City" column
    city_index = data.columns.get_loc("City")

    # Merge the data with city information, excluding "Main Language"
    merged_data = data.merge(df_cities.drop(columns=["Main Language"]), on="City", how="left")

    # Calculate the mean values for missing cities
    mean_values = merged_data[["Population", "Density (per km²)", "Literacy Rate (%)", "Sex Ratio"]].mean()

    # Replace NaN values with the mean of the respective column
    merged_data.fillna(mean_values, inplace=True)

    # Drop the original "City" column
    merged_data.drop(columns=["City"], inplace=True)

    # Reorder columns to place new city-related columns where "City" was
    city_columns = ["Population", "Density (per km²)", "Literacy Rate (%)", "Sex Ratio"]
    cols = merged_data.columns.tolist()

    # Move the new city-related columns to the correct index
    for col in reversed(city_columns):
        cols.insert(city_index, cols.pop(cols.index(col)))

    # Reorder the dataframe
    merged_data = merged_data[cols]

    return merged_data


Pre-processing function:

In [ ]:
def pre_processing(data):

    data = dropId(data)
    data = name_freq(data)
    data = gender_encode(data)
    data = age_integer(data)
    data = academic_pressure(data)
    data = family_mental_illness(data)
    data = suicidal_thoughts(data)
    data = working_or_student(data)
    data = profession(data)
    data = work_pressure(data)
    data = cgpa(data)
    data = satisfaction(data)
    data = sleep_duration(data)
    data = diet(data)
    data = degree(data)
    data = Prefession_categorization(data)
    data = city_one_hot(data)
    data = financial_stress(data)
    return data

training_data = pre_processing(data)
test_data = pre_processing(test)
test_data.to_csv("test_data.csv", index=False)

Here is The function for Normalizing (0-1) the dataset. This is necessary or helpful if we want to use:

| Learning Algorithm |
|--------------------|
| Logistic Regression |
| SVM |
| Neural Network - better |
| K-Means |

In [ ]:
def normalize_dataset(training_data, test_data, columns_to_scale):
    scaler = MinMaxScaler()
    scaler.fit(training_data[columns_to_scale])
    training_data[columns_to_scale] = scaler.transform(training_data[columns_to_scale])
    test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])
    return training_data, test_data

Here is The function for Standardising the dataset. This is necessary or helpful if we want to use:

| Learning Algorithm |
|--------------------|
| Logistic Regression |
| SVM - better |
| Neural Network |
| PCA |

In [ ]:
def standardize_dataset(training_data, test_data, columns_to_scale):
    scaler = StandardScaler()
    scaler.fit(training_data[columns_to_scale])
    training_data[columns_to_scale] = scaler.transform(training_data[columns_to_scale])
    test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])
    return training_data, test_data

The columns that needs to be scaled:

In [ ]:
columns_to_scale = ["Population", "Density (per km²)", "Literacy Rate (%)", "CGPA", "Work/Study Hours"]

def scale(method: str, training_data, test_data):
    if method == "Standardize":
        training_data, test_data = standardize_dataset(training_data, test_data, columns_to_scale)
    else:
        training_data, test_data = normalize_dataset(training_data, test_data, columns_to_scale)
    return training_data, test_data

Splitting the dataset into Train and val, also extracting the labels.

In [ ]:
def split_training_data(seed=0, train_size=0.8, data=training_data):
    np.random.seed(seed)
    X = data.drop(columns=["Depression"])
    y = data["Depression"]
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=train_size, random_state=seed, stratify=y)
    return X_train, X_val, y_train, y_val

Trying out Random forest classifier

Test method to be used anytime you want to try your model. Prints the submission in "submission.csv" in the working directory if you want to submit, but also tests it on the validation dataset

In [ ]:
def test_model(model, test_data, X_val, y_val, test_or_val="val"):
    if test_or_val == "test":
        predictions = model.predict(test_data)

        submission = pd.DataFrame({
            "id": test["id"], 
            "Depression": predictions
        })
        submission.to_csv("submission.csv", index=False)
        print("test successfully executed, results in submission.csv")
    elif test_or_val == "val":
        predictions = model.predict(X_val)
        accuracy = accuracy_score(predictions, y_val)
        print("Accuracy: ", accuracy)


# Random Forest classifier

In [ ]:
def Random_forest(seed, X_train, y_train):
    param_grid = {
        "n_estimators": [50, 100, 200],
        "max_depth": [10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4]
    }

    param_grid = {
        "n_estimators": [200],
        "max_depth": [20],
        "min_samples_split": [5],
        "min_samples_leaf": [1]
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    rf = RandomForestClassifier(random_state=seed)

    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=kf,
        scoring="accuracy",
        n_jobs=-1,  # Use all available CPU cores
        verbose=1   # Shows progress
    )

    # No scaling required

    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_

    print("Best Hyperparameters:", best_params)
    
    model = RandomForestClassifier(max_depth=best_params["max_depth"], 
                            min_samples_leaf=best_params["min_samples_leaf"],
                            min_samples_split=best_params["min_samples_split"],
                            n_estimators=best_params["n_estimators"],
                            class_weight="balanced"
                            )

    model.fit(X_train, y_train)
    
    return model


rf_training_data, rf_test_data = scale("Standardize", training_data, test_data)
X_train, X_val, y_train, y_val = split_training_data(0, data=rf_training_data)
model = Random_forest(0, X_train, y_train)
test_model(model, rf_test_data, X_val, y_val, "val")